In [ ]:
!pip install transformers
from transformers import DistilBertForSequenceClassification,DistilBertTokenizer
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, TensorDataset,DataLoader
from sklearn.model_selection import train_test_split
import torch
from sklearn import metrics

     |████████████████████████████████| 1.9MB 12.8MB/s 
     |████████████████████████████████| 3.2MB 42.3MB/s 
     |████████████████████████████████| 890kB 47.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=9d9803be6069d7ef79e6a21b5369d4d66395518307570618fe4846dcdeee4f78
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=1).to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
train_df = pd.read_csv('binary_train.csv', sep='\t')
val_df = pd.read_csv('binary_val.csv', sep='\t')

In [ ]:
tokenized_utterances = train_df.text.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))
tokenized_utterances_val = val_df.text.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))

# Make sure length of utterances are not greater than 512
for index, utt in enumerate(tokenized_utterances):
    if len(utt) > 512:
      tokenized_utterances[index] = utt[:512]

for index, utt in enumerate(tokenized_utterances_val):
    if len(utt) > 512:
      tokenized_utterances_val[index] = utt[:512]

In [ ]:
# pad sentences for training set
max_len = max(map(len, tokenized_utterances))
print(max_len)
padded_utterances = np.array([i + [0] * (max_len - len(i))  for i in tokenized_utterances])
attention_masked_utterances = np.where(padded_utterances != 0,1,0)

512


In [ ]:
# pad sentences for test set
max_len_val = max(map(len,tokenized_utterances_val))
print(max_len_val)
padded_utterances_val = np.array([i + [0] * (max_len_val -len(i))  for i in tokenized_utterances_val])
attention_masked_utterances_val = np.where(padded_utterances_val != 0,1,0)

512


In [ ]:
# Create tensors
X_train = torch.tensor(padded_utterances, device=device)
X_train_attention = torch.tensor(attention_masked_utterances, device=device)
y_train = torch.tensor(np.array(train_df.sentiment_score.values)[:, np.newaxis], dtype=torch.float32, device=device)

X_val = torch.tensor(padded_utterances_val, device=device)
X_val_attention = torch.tensor(attention_masked_utterances_val, device=device)
y_val = torch.tensor(np.array(val_df.sentiment_score.values)[:, np.newaxis], dtype=torch.float32)


train_data = TensorDataset(X_train, X_train_attention, y_train)
train_loader = DataLoader(train_data,batch_size=16, shuffle=False)

val_dataset = TensorDataset(X_val, X_val_attention)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
# Learning rate tuning (0.015 with 3 epochs performed best)
# Performance on validation set changes each time you train the model, but we saved the best with learning rate 0.015
LRs = [0.015]
NUM_EPOCHS = 3
for LR in LRs:
  # training model with specific learning rate
  print('Training with learning rate of {}'.format(LR))
  optimizer =torch.optim.SGD(model.parameters(), lr=LR)
  loss_fn = torch.nn.BCEWithLogitsLoss()
  for i in range(NUM_EPOCHS):
      print(i)
      model.train()
      batch_number = 0
      loss_list = []
      for X_batch,X_attention_batch,y_batch in train_loader:
          batch_number += 1
          output = model(X_batch,attention_mask=X_attention_batch,labels=None)
          y_pred = output[0]
          loss = loss_fn(y_pred,y_batch)
          loss_list.append(loss)
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
  # evaluate model with specific learning rate
  def sigmoid(x):
    return 1 / (1 + np.exp(-x)) 

  print('Training done, now evaluating')  
  preds = np.zeros([len(val_dataset), 1])
  model.eval()
  for i, (x_batch, x_mask) in enumerate(val_loader):
    outputs = model(x_batch.to(device),attention_mask=x_mask.to(device))

    y_pred = sigmoid(outputs[0].detach().cpu().numpy())

    preds[i*16:(i+1)*16, :] = y_pred

  print(metrics.roc_auc_score(y_val, preds))

  pred_labels = []
  for p in preds:
    if p > 0.5:
      pred_labels.append(1)
    else:
      pred_labels.append(0)
  y_target = list([int(x) for x in y_val])

  print(metrics.classification_report(y_target, pred_labels))

Training with learning rate of 0.015
0
1
2
Training done, now evaluating
0.8038665497515554
              precision    recall  f1-score   support

           0       0.67      0.57      0.61       405
           1       0.82      0.87      0.84       887

    accuracy                           0.78      1292
   macro avg       0.74      0.72      0.73      1292
weighted avg       0.77      0.78      0.77      1292



In [ ]:
# # Save model
# torch.save(model.state_dict(), '80_acc.pt')